# Image Offsets: Create offset arrays used by drizzle

The offset tables created by a previous notebook (Offsets_2) are used to generate the X and Y offset arrays used by drizzle.

Each offset table will generate 2 arrays, for X and Y respectively, stored as FITS image extensions of short float type.

The algorithms were developed in the Timing notebook; here, they are cast as callable functions used in a loop to process all images in the sequence.

In [ ]:
import os, glob
import numpy as np

import multiprocessing as mp
from multiprocessing import Pool

from astropy.table import Table
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans

import rawpy

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

%pylab notebook
%matplotlib notebook

In [ ]:
datadir = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/'

## Functions

In [ ]:
# Comparison functions used to figure out which stars are closest to a given pixel
gt_zero = lambda x: x > 0.0
lt_zero = lambda x: x < 0.0

# Gets the index of the closest star in table. 
# The differences are in the sense pixel - star centroid.
# The comparison functions define from which quadrant the star is drawn from.
def closest(diff_x, diff_y, compare_x, compare_y):
    # Compute mask that masks out everything that is outside 
    # the quadrant defined by the comparison functions
    mask_x = np.where(compare_x(diff_x), 1, 0)
    mask_y = np.where(compare_y(diff_y), 1, 0)
    mask = mask_x * mask_y

    # Get index of star at minimum distance
    distance = np.sqrt((diff_x * diff_x + diff_y * diff_y)) * mask
    if np.nonzero(distance)[0].size > 0:
        mindist = np.min(distance[np.nonzero(distance)])
        index = np.where(distance == mindist )[0][0]
        return index, mindist
    else:
        return -1, 0.0
    
# Parallelization - the offset computation for each individual pixel is prohibitive without parallelization.

class Worker:
    '''
    A class with callable instances that execute the offset calculation
    algorithm over a section of the input image. 
    
    It provides the callable for the `Pool.apply_async` function, and also 
    holds all parameters necessary to perform the calculation.
    
    The 'step' parameters helps save time by allowing the algorithm to work
    only on pixels separated by 'step' (in both X and Y). The remaining pixels
    are filled by interpolation with a 9x9 Gaussian kernel.
    '''
    def __init__(self, x0, y0, size_x, size_y, step_x, step_y, centroid_x, centroid_y, 
                offset_x, offset_y):
        '''
        Parameters:
        
        x0, y0 - top left pixel of the image section designated for this instance
        size_x, size_y - size of the image section
        step_x - step used in the x direction when looping over pixels
        step_y - step used in the y direction when looping over pixels
        centroid_x - 1-D data from the `xcentroid` column in the offsets table 
        centroid_y - 1-D data from the `ycentroid` column in the offsets table 
        offset_x - 1-D data from the `xoffset` column in the offsets table 
        offset_y - 1-D data from the `yoffset` column in the offsets table 
        
        Returns:
        
        dict with output arrays. To be collected by a callback function. 
        '''
        self.x0 = x0
        self.y0 = y0
        self.size_x = size_x
        self.size_y = size_y
        self.step_x = step_x
        self.step_y = step_y

        self.centroid_x = centroid_x
        self.centroid_y = centroid_y
        self.offset_x = offset_x
        self.offset_y = offset_y
        
        # create local output arrays. These have the shape of one single
        # section of the entire image. Once filled up, they are returned 
        # to a callback function that takes care of storing them into
        # the appropriate section of the result arrays.
        self.offset_array_x = np.zeros(shape=(self.size_y, self.size_x))
        self.offset_array_y = np.zeros(shape=(self.size_y, self.size_x))

    def __call__(self):
        for i in range(0, self.size_x, self.step_x):
            for j in range(0, self.size_y, self.step_y):

                pixel_x = int(i + self.x0)
                pixel_y = int(j + self.y0)

                diff_x = pixel_x - self.centroid_x
                diff_y = pixel_y - self.centroid_y

                index = np.array(range(4), dtype=int)
                dist  = np.array(range(4), dtype=float)

                # get index and distance of the closest star, one per quadrant
                index[0], dist[0] = closest(diff_x, diff_y, gt_zero, gt_zero)
                index[1], dist[1] = closest(diff_x, diff_y, lt_zero, gt_zero)
                index[2], dist[2] = closest(diff_x, diff_y, gt_zero, lt_zero)
                index[3], dist[3] = closest(diff_x, diff_y, lt_zero, lt_zero)

                # weighted average of the offset values. The weight is the inverse 
                # distance pixel-star. Beware of zeroed or non-existent distances.
                sumweights = 0.0
                for k in range(len(dist)):
                    if dist[k] > 0.:
                        sumweights += 1./dist[k]
                        
                weighted_offset_x = 0.0
                weighted_offset_y = 0.0

                for k in range(len(index)):
                    if index[k] > 0:
                        weighted_offset_x += self.offset_x[index[k]] * (1./dist[k] / sumweights)
                        weighted_offset_y += self.offset_y[index[k]] * (1./dist[k] / sumweights)

                self.offset_array_x[j][i] = weighted_offset_x
                self.offset_array_y[j][i] = weighted_offset_y
        
        # return the local output arrays with offsets for this section of the image,
        # plus metadata to locate the section on the full offsets arrays.
        return {'x0': self.x0,
                'y0': self.y0,
                'size_x': self.size_x,
                'size_y': self.size_y,
                'offset_array_x': self.offset_array_x,
                'offset_array_y': self.offset_array_y
               }


def compute_offset_arrays(image, offsets_table):
    ''' Function to compute offset arrays with a first pass that parallelizes the
        algorithm, and a second pass that interpolates with convolution.
        
        Parameters:

        image - image array, used to figure out its shape
        offsets_table - astropy Table with offsets for each star (created by script Offsets_2.ipynb)
        
        Returns:

        output arrays with X and Y offsets
    '''
    # get relevant offset information from table
    centroid_x_column = offsets_table['xcentroid'].data
    centroid_y_column = offsets_table['ycentroid'].data 
    offset_x_column   = offsets_table['xoffset'].data 
    offset_y_column   = offsets_table['yoffset'].data    
    
    # this makes indices consistent with daofind-defined centroids
    nx = image.shape[1]
    ny = image.shape[0]

    # work arrays for the parallelized code. These are filled up by
    # the callback function below.
    offset_array_x = np.asarray(image) * 0.0
    offset_array_y = np.asarray(image) * 0.0

    # callback function to collect results from parallel workers
    def collect_result(results):
        rx0 = results['x0']
        ry0 = results['y0']
        sx = results['size_x']
        sy = results['size_y']

        offset_array_section_x = results['offset_array_x']
        offset_array_section_y = results['offset_array_y']

        offset_array_x[ry0:ry0+sy,rx0:rx0+sx] = offset_array_section_x
        offset_array_y[ry0:ry0+sy,rx0:rx0+sx] = offset_array_section_y
    
    # number of processors (optimized for a 10-processor Mac M1 Max)
    nproc = 8

    # To save time, we interpolate on the image array with a stride > 1 over both X and Y. 
    # The remaining pixels are filled in a subsequent step by regular interpolation from 
    # neighboring pixels. Using step=3 and a 9x9 kernel ensures that enough data is used 
    # to interpolate.
    step = 3

    results = []
    pool = Pool(nproc)

    for p in range(nproc):
        # workers are defined over "vertical" sections of an image. Sections span a range
        # of X coordinates, but use the entire range of Y coordinates. Other combinations
        # could be used in case we want a different partitioning in between image sections.
        worker = Worker(int(p*nx/nproc), 0, int(nx/nproc), ny, step, step, 
                        centroid_x_column, centroid_y_column, offset_x_column, offset_y_column)

        r = pool.apply_async(worker, callback=collect_result)
        results.append(r)

    for r in results:
        r.wait()

    pool.close()

    # fill remaining pixels by interpolation
    offset_array_x[offset_array_x == 0.0] = np.nan
    offset_array_y[offset_array_y == 0.0] = np.nan

    kernel = Gaussian2DKernel(x_stddev=1)

    result_x = interpolate_replace_nans(offset_array_x, kernel)
    result_y = interpolate_replace_nans(offset_array_y, kernel)

    result_x[np.isnan(result_x)] = 0.
    result_y[np.isnan(result_y)] = 0.
    
    return result_x, result_y

## Read last table in sequence, and read prototype image

Starting with the last table ensures that we get always the same stars along the entire sequence. Offset tables at the beginning of the sequence may include stars that are dropped later on.

In [ ]:
# last table in sequence
table_list = glob.glob(datadir + '/*.offsets_table.fits')
table_list.sort()
last = table_list[-1]
offsets_table = Table.read(last)

In [ ]:
# prototype image (so the algorithm can know what array size to create)
image_name = last.split('/')[-1]
image_name = image_name.replace('.offsets_table.fits', '.ARW')
image_name = os.path.join(datadir, image_name)
raw = rawpy.imread(image_name)
imarray = raw.raw_image_visible.astype(float)

## Compute offsets

In [ ]:
result_x, result_y = compute_offset_arrays(imarray, offsets_table)

In [ ]:
fig = plt.figure(figsize=(10,11))
a = fig.add_subplot(2, 1, 1)
plt.imshow(result_x)
plt.colorbar()
a.set_title('X offsets')
a1 = fig.add_subplot(2, 1, 2, sharex=a, sharey=a)
plt.imshow(result_y)
plt.colorbar()
_ = a1.set_title('Y offsets')